# AWS GLUE & PYSPARK

###ETL example


In [ ]:
# 1. Import all needed libraries and extensions
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import lower,col,when
import pyspark.sql.functions as f
from awsglue.dynamicframe import DynamicFrame


# 2. Starting a SparkContext and building a GlueContext out of the SparkContext
#    - this is important to work with glue extract and load commands
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session


# 3. This block of code is creating a job from the glue context and initializing before extraction can be performed
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
job = Job(glueContext)
job.init(args['JOB_NAME'], args)



## EXTRACT

# 4. Here the Data is extracted from the s3 bucket from the database and the table name
data = glueContext.create_dynamic_frame.from_catalog(database = "mydata", table_name = "drug200_csv" )



## TRANSFORM

# 5. Here we are converting a dynamic Frame into a pyspark dataframe
#     - After the dataframe is a Pyspark dataframe, perform all filtering and selections specific to what transformation is required
#     - Finally, After transforming the data, I convert back to a Dynamic Frame 
data=data.toDF()
data1=data.filter("age>50")
conditions=when(col("sex")=="F","female").when(col("sex")=="M","male")
data2=data1.withColumn("sex",conditions)
data2= data2.withColumn("bp",f.lower(f.col("bp")))
data2= data2.withColumn("cholesterol",f.lower(f.col("cholesterol")))
data2= data2.withColumn("drug",f.lower(f.col("drug")))
dynamic=DynamicFrame.fromDF(data2,glueContext,'dyf')



## LOAD

# 6. Here the data (dynamic frame) is loaded back into the s3 bucket database mydata and name drug_200.csv
#    - Finally the job is commited and the script will execute
datasink4 = glueContext.write_dynamic_frame.from_catalog(frame = dynamic, database = "mydata", table_name = "drug200_csv")

# Alternate method for writing to s3
# datasink4 = glueContext.write_dynamic_frame.from_options(frame = dynamic, connection_type="s3", connection_options={"path":"s3//aspdiscovery//data"}, format="csv", transformation_ctx="datasink4")
job.commit()


In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

## CREATING SPARK CONTEXT AND ATTACHING GLUE CONTEXT
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

## STARTING JOB BASED ON GLUE CONTEXT
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

## EXTRACTING BOTH ZOMATO & COUNTRY CODE DATASETS
data1 = spark.read.csv("s3://aspdiscovery/CountryCode.csv",header=True,inferSchema=True)
data2 = spark.read.csv("s3://aspdiscovery/zomato.csv",header=True,inferSchema=True)

# visually comparing schema
print(data1.printSchema())
print(data2.printSchema())
print(data1.show(5))
print(data2.show(5))

## TRANSFORMING DATA BY MERGING ON CC
data=data1.join(data2,on="cc")

# converting to a dynamicFrame
dynamic=DynamicFrame(data,glueContext,'fgt')

## LOADING NEW DYNAMIC DATASET BACK TO THE BUCKET
datasink4 = glueContext.write_dynamic_frame.from_catalog(frame = dynamic, database = "data", table_name = "countrycode_csv", transformation_ctx = "datasink4")
job.commit()